#### El presente código fue realizado con la finalidad de cumplir las consideraciones del proyecto.
#### Todo el código fue desarrollado en jupyter
##### con la finalidad de, en esta parte, interactuar con el login, se invita a correr todas las celdas

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
#Se importa la información de la tienda virtual
from lifestore_file import lifestore_products, lifestore_sales, lifestore_searches

In [3]:
#Por comodidad, se trabajará con DataFrames
pd_products = pd.DataFrame(lifestore_products, columns=['id_product', 'name', 'price', 'category', 'stock'])
pd_searches = pd.DataFrame(lifestore_searches, columns=['id_search', 'id product'])
pd_sales = pd.DataFrame(lifestore_sales, columns =['id_sale', 'id_product', 'score', 'date', 'refund'])

### El siguiente código dará solución a la consigna de la tienda virtual LifeStore

# 1.a Productos más vendidos y productos rezagados

#### Productos por ventas

In [4]:
#Se contabilizará el número de ventas por producto
#Se contabilizará el número de devoluciones por producto
#Se calculará el número de ventas reales = ventas-devoluciones
No_prods = len(pd_products['id_product'])#Se obtiene la cantidad de productos diferentes
n_vent = np.zeros(No_prods)              #Se crea un vector donde se almacenará el número de ventas por producto
n_dev = np.zeros(No_prods)               #Se crea un vector donde se almacenará el número de devoluciones por producto
for i in range(len(pd_sales)):
    #Se registra el número de compras por productos
    n_vent[pd_sales['id_product'][i]-1] += 1
    #Se registra el número de devoluciones por productos
    if pd_sales['refund'][i] == 1:
        n_dev[pd_sales['id_product'][i]-1] += 1

#Se calculan las ventas reales
n_vent_tot = n_vent - n_dev

In [5]:
#Se ordena el arreglo de cantidad vendida neta
n_vent_sort = sorted(n_vent_tot, reverse=True)

In [6]:
No_prod_vent = 5     #Se especifica la cantidad de productos con mayores ventas a mostrar en el listado

data_mv = []#Se crea un array

#Se utilizan while en lugar de for para considerar los casos de productos con misma cantidad de ventas
i = 0; j = 0
while i+j <= No_prod_vent:
    position = np.where(n_vent_tot==n_vent_sort[i])
    j = 0
    while j < len(position[0]) and j+i < No_prod_vent:
        data_mv.append([pd_products['id_product'][position[0][j]], pd_products['name'][position[0][j]], pd_products['category'][position[0][j]], n_vent_sort[i]])
        j+=1
    i+=1

#Se convierte en DataFrame para lucir la tabla jejeje
pd_mv = pd.DataFrame(data_mv, columns=['ID', 'Nombre', 'Categoría', 'Número de ventas'], index=[i+1 for i in range(No_prod_vent)])


#### Productos por búsquedas

In [7]:
#Se contabilizará el número de búsquedas por producto
No_prods = len(pd_products['id_product'])#Se obtiene la cantidad de productos diferentes
n_busq = np.zeros(No_prods)              #Se crea un vector donde se almacenará el número de búsquedas por producto

for i in range(len(pd_searches)):
    #Se registra el número de búsquedas por productos
    n_busq[pd_searches['id product'][i]-1] += 1

In [8]:
##Se ordena el arreglo de búsquedas por producto
n_busq_sort = sorted(n_busq, reverse=True)

In [9]:
No_prod_busq = 10    #Se especifica la cantidad de productos con mayores búsquedas a mostrar en el listado

data_ms = []#Se crea un array

#Se utilizan while en lugar de for para considerar los casos de productos con misma cantidad de ventas
i = 0; j = 0
while i+j <= No_prod_busq:
    position = np.where(n_busq==n_busq_sort[i])
    j = 0
    while j < len(position[0]) and j+i < No_prod_busq:
        data_ms.append([pd_products['id_product'][position[0][j]], pd_products['name'][position[0][j]], pd_products['category'][position[0][j]], n_busq_sort[i]])
        j+=1
    i+=1

#Se convierte en DataFrame para lucir la tabla jejeje
pd_ms = pd.DataFrame(data_ms, columns=['ID', 'Nombre', 'Categoría', 'Número de búsquedas'], index=[i+1 for i in range(No_prod_busq)])


# 1.b Productos más vendidos y productos rezagados

#### Productos por categorías

In [10]:
#Se obtienen las categorías diferentes
Categorias = list(set(pd_products['category']))

In [11]:
#Se hará un diccionario donde, por categoría, se tendrán los id_product que los componen
#así como el total de ventas, total de devoluciones, ventas netas y total de búsquedas de cada id_product
dic_categ = {}

for categoria in Categorias:
    #Se encuentran todos los productos pertenecientes a "categoria"
    productosCat = pd_products[pd_products['category']==categoria]
    #Se crea un arreglo que contendrá la información de cada producto
    prods_cat = []
    for producto in range(len(productosCat)):
        #Se obtienen los id_product de los productos
        id_prod_Cat = productosCat.iloc[producto, 0]
        #Se calcula las ventas de cada producto
        Ventas_prod = pd_sales[pd_sales['id_product']==id_prod_Cat]
        NoVentas = len(Ventas_prod)
        #Se calcula las devoluciones de cada producto
        NoDevoluciones = len(Ventas_prod[Ventas_prod['refund']==1])
        #Se calcula las ventas netas de cada producto
        Ventas_net = NoVentas - NoDevoluciones
        #Se calculan las búsquedas de cada producto
        Busquedas_prod = pd_searches[pd_searches['id product']==id_prod_Cat]
        NoBusquedas = len(Busquedas_prod)
        #Se agrega la información al arreglo
        dic_prod = {'id': id_prod_Cat, 'Ventas brutas': NoVentas, 'Devoluciones': NoDevoluciones, 'Ventas netas': Ventas_net, 'Busquedas': NoBusquedas}
        prods_cat.append(dic_prod)
    dic_categ[categoria] = prods_cat

In [12]:
#Se obtiene la cantidad de productos por categorías
cantidad = [len(pd_products[pd_products['category']==categoria]) for categoria in Categorias]

productos_vendidos = []
productos_buscados = []

for categoria in Categorias:
    #Se crea un DataFrame de cada categoria
    df_categ = pd.DataFrame(dic_categ[categoria])
    prod_vend = len(df_categ[df_categ['Ventas netas']>0])
    prdo_busq = len(df_categ[df_categ['Busquedas']>0])
    productos_vendidos.append(prod_vend)
    productos_buscados.append(prdo_busq)

# 2. Productos por reseña en el servicio

In [13]:
#Se hará un diccionario donde, para cada producto, identificado por su ID, se agregarán los puntajes dados, 
#el número de reseñas y el promedio de reseñas
dic_score = {}

#Se hará un registro para cada ID
for idx in pd_products['id_product']:
    #Se obtienen la información de cada producto
    id_info = pd_sales[pd_sales['id_product']==idx]
    #Nos quedamos sólo con los score
    scores = id_info['score']
    #Valores a guardar
    name = pd_products.iloc[idx-1, 1]
    vec_scores = scores.tolist()
    no_scores = len(scores)
    val_scores = np.mean(scores)
    dic_score[idx] = {'Nombre':name, 'Reseñas':vec_scores, 'Total de reseñas':no_scores, 'Puntaje':val_scores}

In [14]:
#Se convierte el diccionario en dataframe donde cada ID de producto será una fila
df_score = pd.DataFrame.from_dict(dic_score, orient='index')

In [15]:
#Se ordena el DataFrame con respecto a la media de reseñas con orden descendente
df_sorted_media = df_score.sort_values(by=['Puntaje', 'Total de reseñas'], ascending=False)

# 3. Reporte financiero

In [16]:
df_sales = pd_sales.loc[:, ['id_product', 'refund', 'date']]
df_sales['date'] = pd.to_datetime(df_sales['date'], format='%d/%m/%Y')
df_sales = df_sales.sort_values('date')

In [17]:
init_date = min(df_sales['date'])
end_date = max(df_sales['date'])

#Se crea una lista de fechas mensuales desde la fecha inicial hasta la fecha final
Fechas = pd.date_range(init_date.replace(day=1), end_date.replace(day=1), 
              freq='MS').strftime("%Y-%m").tolist()

In [18]:
#Se creará un diccionario que contenga:
#fecha (formato mes/Año), ventas brutas (ventas realizadas sin importar devoluciones)
#ganancias netas (ganancias descontando devoluciones), ventas promedio mensuales (ganancias netas/ventas netas)
ingresos = {}

for idx, fecha in enumerate(Fechas):
    #Ventas por mes    
    if idx==len(Fechas)-1:
        ventas_mes = df_sales[df_sales['date']>=Fechas[-1]]
    else:
        ventas_mes = df_sales[(df_sales['date']<Fechas[idx+1]) & (df_sales['date']>=Fechas[idx])]
    #Cantidad de ventas realizadas
    ventas_brutas = len(ventas_mes)
    #Cantidad de ventas realizadas descontando devoluciones
    vent_net = ventas_mes[ventas_mes['refund']==0]
    ventas_netas = len(vent_net)
    ganancias = 0
    for i in range(ventas_netas):
        #Se obtiene el ID del producto vendido
        idx = vent_net.iloc[i, 0]
        ganancias+=pd_products.iloc[idx-1, 2]
    if ventas_netas==0:
        ventas_prom=0
    else:
        ventas_prom = ganancias/ventas_netas
    ingresos[fecha] = {'Ventas':ventas_brutas, 'Ganancias':ganancias, 'Ventas promedio':ventas_prom}

In [19]:
df_ingresos = pd.DataFrame.from_dict(ingresos, orient='index')

# Interacción con el usuario: Login

In [20]:
#Se lee el archivo csv de usuarios
#Los usuarios existentes son:
#Name: Jimmy             Pass: el_mejor
#Name: Fernando          Pass: merece10
file = 'usuarios.csv'
usuarios = pd.read_csv(file)

In [21]:
usuarioAccedio = False
intentos = 0
check = False

# Recibo constantemente sus intentos
while not usuarioAccedio:
    # Primero ingresa Nombre de usuario
    name = input('Introduce el nombre de usuario:\n')
    password = input('Introduce la contraseña:\n')
    intentos += 1
    # Reviso si el nombre está en el archivo
    try:
        idx = usuarios[usuarios['Name']==name].index.item()
        check = usuarios.iloc[idx,1]==password
    except:
        print('')
    
    if check:
        usuarioAccedio = True
        print('Bienvenido, se desplegará la información a continuación!')
        break
    else:
        print('Error en el nombre de usuario y/o contraseña')
        print(f'Tienes {3 - intentos} intentos restantes')
            
    if intentos == 3:
        print('Alcanzaste el límite de intentos, adios!')
        break

#############################################################################
########## código en caso de acceso correcto por el usuario  ################
#############################################################################
        
#Si hubo acceso correcto, se deplegará la información de cada sección
if check:
    print("\033[1m" + '\nProductos con mayores ventas:' + "\033[0m")
    print('ID\tCategoría\tNúmero de ventas\tNombre')
    for idx in range(len(pd_mv)):
        id_ = pd_mv.iloc[idx, 0]
        nombre = pd_mv.iloc[idx, 1]
        categoria = pd_mv.iloc[idx, 2]
        noventas = pd_mv.iloc[idx, 3]
        print(f'{id_}\t{categoria}\t{noventas}\t{nombre}')
    
    print("\033[1m" + '\nProductos con mayores búsquedas:' + "\033[0m")
    print('ID\tCategoría\tNúmero de ventas\tNombre')
    for idx in range(len(pd_ms)):
        id_ = pd_ms.iloc[idx, 0]
        nombre = pd_ms.iloc[idx, 1]
        categoria = pd_ms.iloc[idx, 2]
        nobusq = pd_ms.iloc[idx, 3]
        print(f'{id_}\t{categoria}\t{nobusq}\t{nombre}')
    
    ##### Productos con menores venta para cada categoría    
    print("\033[1m" + '\nLista de los 5 productos con menores ventas por categoría:' + "\033[0m")
    Noproductos = 5
    for categoria in Categorias:
        #Se crea un DataFrame de cada categoria
        df_categ = pd.DataFrame(dic_categ[categoria])
        #Se ordena de menor a mayor el DataFrame con respecto a Ventas netas
        df_sorted_cat = df_categ.sort_values(by='Ventas netas')
        print("\033[1m" + 'Categoría:' + categoria + "\033[0m")
        print('ID\t Ventas netas\t Nombre')
        item = 0
        while item < Noproductos and item < len(df_sorted_cat):
            id_ = df_sorted_cat.iloc[item, 4]
            idx = pd_products[pd_products['id_product']==id_].index.item()
            name = pd_products.iloc[idx, 1]
            ventas = df_sorted_cat.iloc[item, 3]
            print(id_, '\t', ventas, '\t', name)
            item+=1
#         print('\n')
    
    #### Productos con menores búsquedas para cada categoría
    print("\033[1m" + '\nLista de los 10 productos con menores búsquedas por categoría:' + "\033[0m")
    Noproductos = 10
    for categoria in Categorias:
        #Se crea un DataFrame de cada categoria
        df_categ = pd.DataFrame(dic_categ[categoria])
        #Se ordena de menor a mayor el DataFrame con respecto a Busquedas
        df_sorted_cat = df_categ.sort_values(by='Busquedas')
        print("\033[1m" + 'Categoría:' + categoria + "\033[0m")
        print('ID\t Búsquedas\t Nombre')
        item = 0
        while item < Noproductos and item < len(df_sorted_cat):
            id_ = df_sorted_cat.iloc[item, 4]
            idx = pd_products[pd_products['id_product']==id_].index.item()
            name = pd_products.iloc[idx, 1]
            busquedas = df_sorted_cat.iloc[item, 0]
            print(id_, '\t', busquedas, '\t', name)
            item+=1
#         print('\n')
        
    #### Productos con las mejores reseñas
    Noproductos = 5
    print("\033[1m" + '\nProductos con las mejores reseñas:' + "\033[0m")
    print('ID\tPuntaje\tNombre')
    for i in range(Noproductos):
        id_val = df_sorted_media.index[i]
        name_val = df_sorted_media.iloc[i, 0] #pd_products.iloc[id_val, 1]
        score_val = df_sorted_media.iloc[i, 3]
        print(f'{id_val}\t{score_val}\t{name_val}')
        
    #### Productos con las peores reseñas
    Noproductos = 5
    print("\033[1m" + '\nProductos con las peores reseñas:' + "\033[0m")
    print('ID\tPuntaje\tNombre')
    for i in range(Noproductos):
        id_val = df_sorted_media[df_sorted_media['Total de reseñas']>0].index[-1-i]
        name_val = df_sorted_media[df_sorted_media['Total de reseñas']>0].iloc[-1-i, 0]
        score_val = df_sorted_media[df_sorted_media['Total de reseñas']>0].iloc[-1-i, 3]
        print(f'{id_val}\t{round(score_val, 1)}\t{name_val}')
        
    ####Ingreso total y anual 2020
    print("\033[1m" + '\nREPORTE FINANCIERO' + "\033[0m")
    print("\033[1m" + 'Ingreso Total' + "\033[0m" + ' = {}'.format(np.sum(df_ingresos['Ganancias'])))
    print("\033[1m" + 'Ingreso Anual 2020' + "\033[0m" + ' = {}'.format(np.sum(df_ingresos['Ganancias'])))
    #### Ventas mensuales
    print("\033[1m" + 'Reporte de ingresos mensuales:' + "\033[0m")
    print('Fecha\tVentas\tGanancias\tVentas promedio')
    for idx in range(len(df_ingresos)):
        mes = df_ingresos.index[idx]
        venta = df_ingresos.iloc[idx, 0]
        ganancia = df_ingresos.iloc[idx, 1]
        ventas_prom = df_ingresos.iloc[idx, 2]
        print(f'{mes}\t{venta}\t{ganancia}\t\t{ventas_prom}')

Introduce el nombre de usuario:
Fernando
Introduce la contraseña:
merece10
Bienvenido, se desplegará la información a continuación!

Productos con mayores ventas:
ID	Categoría	Número de ventas	Nombre
54	discos duros	49.0	SSD Kingston A400, 120GB, SATA III, 2.5'', 7mm
3	procesadores	42.0	Procesador AMD Ryzen 5 2600, S-AM4, 3.40GHz, Six-Core, 16MB L3 Cache, con Disipador Wraith Stealth
5	procesadores	20.0	Procesador Intel Core i3-9100F, S-1151, 3.60GHz, Quad-Core, 6MB Cache (9na. Generación - Coffee Lake)
42	tarjetas madre	18.0	Tarjeta Madre ASRock Micro ATX B450M Steel Legend, S-AM4, AMD B450, HDMI, 64GB DDR4 para AMD
57	discos duros	15.0	SSD Adata Ultimate SU800, 256GB, SATA III, 2.5'', 7mm

Productos con mayores búsquedas:
ID	Categoría	Número de ventas	Nombre
54	discos duros	263.0	SSD Kingston A400, 120GB, SATA III, 2.5'', 7mm
57	discos duros	107.0	SSD Adata Ultimate SU800, 256GB, SATA III, 2.5'', 7mm
29	tarjetas madre	60.0	Tarjeta Madre ASUS micro ATX TUF B450M-PLUS GAMING, S-AM4, AM

In [22]:
#Para ver las gráficas mostradas en el reporte escrito, se invita a revisar los archivos adicionales